In [47]:
%reload_ext autoreload
%autoreload 2


import matplotlib.pyplot as plt
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

from util.fusion_util import *
from util.data_util import *
from util.general_util import *
from sites.config import *
from model.pdr import PDR
from model.bayes_filter import *

In [48]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

np.set_printoptions(threshold=10)

In [49]:
site = MtrCtl()
site_name = site.name
constraints1 = site.constraints1
constraints2 = site.constraints2
map_path = os.path.join('..', site.map_path)
dataset_dir = os.path.join('..', site.dataset_dir)
log_dir = os.path.join('..', site.log_dir, 'ble')

In [50]:
gridsize = site.gridsize
window_size = 7000
batch_size = 1024
n_epochs = 500
n_layers = 1
d = 8

In [56]:
rp_locs = generate_line_grid(constraints1, gridsize) + generate_filled_grid(constraints2, gridsize)
fp_features, fp_locs, beacon_list1 = read_ble_fingerprint(glob.glob(os.path.join(dataset_dir, 'm*_Ble.txt')), window_size=window_size)
test_features, test_locs, beacon_list2 = read_ble_fingerprint(glob.glob(os.path.join(dataset_dir, 'n*_Ble.txt')), window_size=window_size)

with open(os.path.join(log_dir, 'preprocessor'), 'rb') as f:
    preprocessor = pickle.load(f)

fp_features = preprocessor.preprocess(fp_features, beacon_list1).astype(np.float32)
test_features = preprocessor.preprocess(test_features, beacon_list2).astype(np.float32)
fp_features = fp_features[np.sum(fp_features, axis=1) != 0]
test_features = test_features[np.sum(test_features, axis=1) != 0]
print(fp_features.shape, test_features.shape)

rp_features = compute_reference_features(rp_locs, fp_features, fp_locs, gridsize, p=10).astype(np.float32)

(783, 153) (169, 153)


In [57]:
j = 60
i = get_closest_grid(fp_locs[j], rp_locs)
print(rp_features[i], fp_features[j])

[0.0000000e+00 1.2219312e-03 1.5330410e-03 ... 1.3004799e-03 3.7141039e-05
 1.1165949e-02] [0. 0. 0. ... 0. 0. 0.]


In [58]:
iii = np.nonzero(fp_features[j])[0]

In [67]:
np.exp(-np.linalg.norm(rp_features[:,iii] - fp_features[j][iii], axis=1)**2 / 0.5)

array([0.76739246, 0.76739246, 0.76739246, ..., 0.87807107, 0.76841736,
       0.7785202 ], dtype=float32)

In [171]:
np.mean(np.exp(-(rp_features[:,np.nonzero(fp_features[j])] - fp_features[j][np.nonzero(fp_features[j])])**2 / 1e-1), axis=2).ravel()

array([0.56880647, 0.56880647, 0.56880647, ..., 0.7530123 , 0.56880647,
       0.56880647], dtype=float32)

In [113]:
rp_locs, ble_rpf = read_reference_points(os.path.join(log_dir, 'grid.txt'))
ble, ble_w = load_tf_model(os.path.join(log_dir))

In [114]:
ble.predict([[0 for i in range(153)]])

<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[ 3.6857183, -2.6237242, -2.3890076, -2.9441853,  2.1938772,
         1.7918653,  2.2456832, -2.053021 ]], dtype=float32)>

In [ ]:
ble.predict(ble_rpf)

In [15]:
t = 0.1
F = np.eye(4)
F[0][2] = F[1][3] = t

B = np.array([[0.5*t*t, 0], [0, 0.5*t*t], [t, 0], [0, t]])
u = np.array([0.01, 0.01])

In [24]:
kf = KalmanFilter(x=[0,0,0,0], P=np.eye(4), H=np.eye(4), )

In [ ]:
loc = [1,1]
speed = [0,0]



In [44]:
kf._predict(F=F, B=B, u=u, Q=np.eye(4))

In [46]:
kf._update(z=[1,2,0,0], R=np.eye(4))

In [38]:
kf.x

array([9.93336933e-01, 1.98663539e+00, 1.60590980e-03, 2.60217137e-03])

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

array([[0., 0.],
       [0., 0.]])